In [15]:
#Importing all packages needed 
#1) Fundatmental 
import pandas as pd
import numpy as np

#2) Preprocessing 
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
import imblearn
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SMOTENC

#3) Model 
from sklearn.ensemble import RandomForestClassifier

#4) Validation  
from sklearn.model_selection import LeaveOneOut

#5) Performance metrics  
from sklearn.metrics import roc_auc_score, confusion_matrix,accuracy_score

#6) Custom Functions
#!pip install ipynb
from ipynb.fs.full.Functions import *


import warnings
warnings.filterwarnings("ignore")

#  Synthetic Minority Oversampling Technique (SMOTE)

# 1. Tapping Features

The only features that showed evidence of being sensitive enough to identify *Preclinical* subjects were the tapping features. However we saw that the sensitivity was low at 0.550 due to the imbalance nature of the dataset and thus we apply SMOTE to make the dataset balanced. 

The performnace metrics improve thanks to SMOTE with an accuracy of 0.797 and AUC of 0.797, compared to the model without SMOTE that had an accuracy of 0.772 and AUC of 0.721. The sensitivity saw an improvement with a value of 0.811 compared to 0.550 without SMOTE. Thus we can conclude that SMOTE has been successful at improving the imbalanced and subsequently the performance

In [19]:
#Reading in the dataset
data=pd.read_spss("FTT.sav")  

#Taking the columns that are needed 
data_class=data[['groups','P_TAU','AB42','FTT_rate','FTT_variability_total_Log10','age', 'gender','education','AL','AT','RD','RDT','BNT','ANIMALES','VOSP_NUM','TMT_A','TMT_B','STROOP_LECT','STROOP_COLOR','STROOP_INT','SDMT',"PL_FTT_time_interval"]]

#Renaming the group name to biodiag to match the previous dataset1 and numbering also in the same way
data_class = data_class.rename(columns={'groups': 'biodiag'})
data_class['biodiag'] = data_class['biodiag'].replace({"grupo control":1, "grupo preclinico":2, "grupo con EA":3})

#Turning gender into binary 
data_class['gender'] = data_class['gender'].replace({"male":1, "female":0})


#Creating a dataframe for tapping features only
feat_select=data_class[["biodiag","FTT_variability_total_Log10","PL_FTT_time_interval","FTT_rate"]]

In [20]:
#Preparing the data
#dropping the biomarkers that should not be used in this model
#feat_select=data_class.drop(["P_TAU","AB42","STROOP_LECT","gender","ANIMALES","RDT","VOSP_NUM","education","AL","RD","BNT","AT","TMT_A"],axis=1)
feat_select=data_class[["biodiag","FTT_variability_total_Log10","PL_FTT_time_interval","FTT_rate"]]

#1)Preclinical vs Control 
#Keeping only the subject in the control(1) and preclinical groups(2)
data_model=feat_select.loc[feat_select['biodiag'].isin([1,2])]
#Calling the function data_prep to further prepare the dataset:
#One hot encodes the group variable (biodiag)
#Splits the data into X and y, and turns then into the desired format, an array
X,y,coulmnNames=data_prep(data_model)
#Oversampling the dataset with SMOTE
X,y= smote_data(SMOTE,None,X, y)

#Call the function featureimportance to run the model
acc, roc_auc, featureImportance,sensitivity,specificity=featureimportance(X,y,RandomForestClassifier(random_state=1),MinMaxScaler,KNNImputer)
print("For control vs AD preclinical the accuracy is %.3f and the auc is %.3f" % (acc, roc_auc))
print("For control vs AD preclinical the sensitivity is %.3f and the specificity is %.3f" % (sensitivity, specificity))
print_importance(featureImportance,coulmnNames)


For control vs AD preclinical the accuracy is 0.797 and the auc is 0.797
For control vs AD preclinical the sensitivity is 0.811 and the specificity is 0.784
                       feature  importance
1         PL_FTT_time_interval    0.346775
0  FTT_variability_total_Log10    0.329076
2                     FTT_rate    0.324148
